In [3]:
import cv2
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import zipfile

cv2.__version__

'4.10.0'

In [5]:
import tensorflow
tensorflow.__version__

2024-12-02 14:04:10.308514: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 14:04:10.338180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733159050.370284   59102 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733159050.379541   59102 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 14:04:10.409275: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

'2.18.0'

In [6]:
from tensorflow.keras.models import load_model

diretorio = './modelo_01_expressoes.keras'
model = load_model(diretorio)

I0000 00:00:1733159055.267457   59102 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2278 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [9]:
arquivo_video = "./Material/Videos/video_teste04.mp4"
cap = cv2.VideoCapture(arquivo_video)

conectado, video = cap.read()
print(video.shape) 

(480, 640, 3)


In [11]:
redimensionar = True
largura_maxima = 600  

if (redimensionar and video.shape[1]>largura_maxima):
  proporcao = video.shape[1] / video.shape[0]
  video_largura = largura_maxima
  video_altura = int(video_largura / proporcao)
else:
  video_largura = video.shape[1]
  video_altura = video.shape[0]

In [15]:
# nome do arquivo de vídeo que será salvo
nome_arquivo = './resultado_video_teste04.avi'

# definição do codec
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# FourCC é um código de 4 bytes usado para especificar o codec de vídeo. A lista de códigos disponíveis pode ser encontrada no site fourcc.org
# Codecs mais usados: XVID, MP4V, MJPG, DIVX, X264...
# Por exemplo, para salvar em formato mp4 utiliza-se o codec mp4v (o nome do arquivo também precisa possuir a extensão .mp4)
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# FPS - frames por segundo
fps = 24
# se quiser deixar o video um pouco mais lento pode diminuir o numero de frames por segundo para 20

saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))

In [16]:
from tensorflow.keras.preprocessing.image import img_to_array
from IPython.display import display

haarcascade_faces = './haarcascade_frontalface_default.xml' # arquivo haarcascade

# define os tamanhos para as fontes
fonte_pequena, fonte_media = 0.4, 0.7

fonte = cv2.FONT_HERSHEY_SIMPLEX

expressoes = ["Raiva", "Nojo", "Medo", "Feliz", "Triste", "Surpreso", "Neutro"]

while (cv2.waitKey(1) < 0):
    conectado, frame = cap.read()

    if not conectado:
        break  # se ocorreu um problema ao carregar a imagem então interrompe o programa

    t = time.time() # tempo atual, antes de iniciar (vamos utilizar para calcular quanto tempo levou para executar as operações)

    # frame_video = np.copy(frame) # faz uma copia do frame do video

    if redimensionar: # se redimensionar = True então redimensiona o frame para os novos tamanhos
      frame = cv2.resize(frame, (video_largura, video_altura))

    face_cascade = cv2.CascadeClassifier(haarcascade_faces)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # converte pra grayscale
    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.2, minNeighbors=5,minSize=(30,30))

    if len(faces) > 0:
        for (x, y, w, h) in faces:

            frame = cv2.rectangle(frame,(x,y),(x+w,y+h+10),(255,50,50),2) # desenha retângulo ao redor da face

            roi = gray[y:y + h, x:x + w]      # extrai apenas a região de interesse (ROI) que é onde contém o rosto
            roi = cv2.resize(roi, (48, 48))    # antes de passar pra rede neural redimensiona para o tamanho das imagens de treinamento
            roi = roi.astype("float") / 255.0  # normaliza
            roi = img_to_array(roi)            # converte para array para que a rede possa processar
            roi = np.expand_dims(roi, axis=0)  # muda o shape da array

            # faz a predição - calcula as probabilidades
            result = model.predict(roi)[0]
            print(result)

            if result is not None:
                resultado = np.argmax(result) # encontra a emoção com maior probabilidade
                cv2.putText(frame,expressoes[resultado],(x,y-10), fonte, fonte_media,(255,255,255),1,cv2.LINE_AA) # escreve a emoção acima do rosto

    # tempo processado = tempo atual (time.time()) - tempo inicial (t)
    cv2.putText(frame, " frame processado em {:.2f} segundos".format(time.time() - t), (20, video_altura-20), fonte, fonte_pequena, (250, 250, 250), 0, lineType=cv2.LINE_AA)

    saida_video.write(frame) # grava o frame atual

print("Terminou")
saida_video.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
[1.08973667e-01 5.37459680e-04 2.14964151e-01 3.25772422e-03
 5.62625647e-01 1.22351036e-03 1.08417846e-01]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
[5.8866121e-02 5.4347460e-05 1.8620940e-01 1.0125517e-03 6.8002588e-01
 2.1072617e-04 7.3621020e-02]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
[4.8460212e-02 1.4076604e-05 1.5409954e-01 6.5033679e-04 7.1697050e-01
 8.7200766e-05 7.9718105e-02]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
[4.3862190e-02 1.8392233e-05 1.6715148e-01 5.5132358e-04 7.3132437e-01
 8.5411557e-05 5.7006828e-02]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
[4.9317889e-02 2.0934027e-05 1.5764867e-01 7.1619701e-04 7.2775412e-01
 8.6796601e-05 6.4455383e-02]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
[7.0059724e-02 1.2315586e-04 2.0532130e-01 1.1536330e-03 6.3104951e-01
 4.3418675e-04 9.1858432e-02]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
[4.0911872e-02 1.8316319e-05 1.6323505e-01 3.8945809e-04 7.5693190e-01
 5.1281284e-05 3.8462158e-02]
1/1 ━━━━━━━━━━━━━━━━

KeyboardInterrupt: 